<a href="https://colab.research.google.com/github/jiteshpabla/Meal-Detection-using-CGM-data-/blob/main/Meal_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

In [ ]:
df_cgm = pd.read_csv("CGMData.csv")
df_cgm = df_cgm.dropna(axis='columns', how="all")
df_cgm

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Index,Date,Time,Sensor Glucose (mg/dL),ISIG Value,Event Marker,Sensor Exception
0,20355,2/12/2018,13:22:27,118.0,15.16,NaN,NaN
1,20356,2/12/2018,13:17:27,122.0,16.12,NaN,NaN
2,20357,2/12/2018,13:12:27,NaN,NaN,NaN,SENSOR_ERROR
3,20358,2/12/2018,13:07:27,NaN,NaN,NaN,SENSOR_ERROR
4,20359,2/12/2018,13:02:27,NaN,NaN,NaN,SENSOR_ERROR
...,...,...,...,...,...,...,...
55338,52855,7/25/2017,12:28:54,311.0,44.91,NaN,NaN
55339,52856,7/25/2017,12:23:54,311.0,44.26,NaN,NaN
55340,52857,7/25/2017,12:18:54,309.0,44.51,NaN,NaN
55341,52858,7/25/2017,12:13:54,310.0,45.16,NaN,NaN


In [ ]:
df_insulin = pd.read_csv("InsulinData.csv")
df_insulin = df_insulin.dropna(axis='columns', how="all")
df_insulin

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (13,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Bolus Duration (h:mm:ss),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason
0,0,2/12/2018,13:20:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TEMPORARY,NaN,NaN
1,1,2/12/2018,13:20:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other Alarm Code: 112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2/12/2018,13:18:48,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2/12/2018,13:18:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USER_SUSPEND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2/12/2018,13:12:33,NaN,NaN,NaN,Normal,0.03,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41430,22135,7/24/2017,19:00:01,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41431,22136,7/24/2017,18:59:44,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41432,22137,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL_PUMPING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41433,22138,7/24/2017,18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rewind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Convert to data time

In [ ]:
def convert_datetime(df, only_date=False):
    df['Date']= pd.to_datetime(df['Date']) 
    if not only_date:
        df['Time']= pd.to_datetime(df['Time'])
    return

In [ ]:
convert_datetime(df_insulin)
convert_datetime(df_cgm)

In [ ]:
def add_timestamp(df, timeCol_has_no_date = False): #true for df_cgm2
  df["timestamp"] = None
  i = 0
  for idx, row in df.iterrows():
    #print(row["Date"], "   ", row["Time"])
    #print(str(row["Date"])[:-9]+str(row["Time"])[-9:])
    if not timeCol_has_no_date:
      df.at[idx,'timestamp'] = pd.Timestamp(str(row["Date"])[:-9]+str(row["Time"])[-9:])
    else:
      df.at[idx,'timestamp'] = pd.Timestamp(str(row["Date"])[:-9]+" "+str(row["Time"]))
  #df["timestamp"] = "aaaa"

In [ ]:
add_timestamp(df_cgm)
df_cgm

,Index,Date,Time,Sensor Glucose (mg/dL),ISIG Value,Event Marker,Sensor Exception,timestamp
0,20355,2018-02-12,2020-11-24 13:22:27,118.0,15.16,NaN,NaN,2018-02-12 13:22:27
1,20356,2018-02-12,2020-11-24 13:17:27,122.0,16.12,NaN,NaN,2018-02-12 13:17:27
2,20357,2018-02-12,2020-11-24 13:12:27,NaN,NaN,NaN,SENSOR_ERROR,2018-02-12 13:12:27
3,20358,2018-02-12,2020-11-24 13:07:27,NaN,NaN,NaN,SENSOR_ERROR,2018-02-12 13:07:27
4,20359,2018-02-12,2020-11-24 13:02:27,NaN,NaN,NaN,SENSOR_ERROR,2018-02-12 13:02:27
...,...,...,...,...,...,...,...,...
55338,52855,2017-07-25,2020-11-24 12:28:54,311.0,44.91,NaN,NaN,2017-07-25 12:28:54
55339,52856,2017-07-25,2020-11-24 12:23:54,311.0,44.26,NaN,NaN,2017-07-25 12:23:54
55340,52857,2017-07-25,2020-11-24 12:18:54,309.0,44.51,NaN,NaN,2017-07-25 12:18:54
55341,52858,2017-07-25,2020-11-24 12:13:54,310.0,45.16,NaN,NaN,2017-07-25 12:13:54


In [ ]:
#add_timestamp(df_insulin2, True)
add_timestamp(df_insulin)

In [ ]:
df_insulin

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Bolus Duration (h:mm:ss),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
0,0,2018-02-12,2020-11-24 13:20:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TEMPORARY,NaN,NaN,2018-02-12 13:20:53
1,1,2018-02-12,2020-11-24 13:20:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other Alarm Code: 112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:20:48
2,2,2018-02-12,2020-11-24 13:18:48,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:18:48
3,3,2018-02-12,2020-11-24 13:18:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,USER_SUSPEND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 13:18:48
4,4,2018-02-12,2020-11-24 13:12:33,NaN,NaN,NaN,Normal,0.03,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,NaN,NaN,NaN,NaN,NaN,CLOSED_LOOP_MICRO_BOLUS,NaN,NaN,NaN,2018-02-12 13:12:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41430,22135,2017-07-24,2020-11-24 19:00:01,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 19:00:01
41431,22136,2017-07-24,2020-11-24 18:59:44,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41432,22137,2017-07-24,2020-11-24 18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL_PUMPING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44
41433,22138,2017-07-24,2020-11-24 18:59:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rewind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-24 18:59:44


# Get meal data

In [ ]:
from datetime import timedelta, datetime

In [ ]:
#get meal points (NON ZERO POINTS)
meal_point = df_insulin.query("`BWZ Carb Input (grams)` >0")

meal_point

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Bolus Duration (h:mm:ss),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
48,48,2018-02-12,2020-11-24 09:15:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,NaN,13.0,NaN,34.0,153.0,0.05,2.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 09:15:45
129,129,2018-02-12,2020-11-24 02:30:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2,NaN,NaN,12.0,NaN,15.0,0.0,0.00,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 02:30:55
188,188,2018-02-11,2020-11-24 20:33:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8,NaN,NaN,8.0,NaN,71.0,0.0,0.00,8.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 20:33:18
207,207,2018-02-11,2020-11-24 18:14:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,115.0,80.0,8.0,100.0,8.0,0.0,0.00,1.00,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 18:14:37
222,222,2018-02-11,2020-11-24 16:27:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,10.0,NaN,40.0,0.0,0.00,4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 16:27:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41265,21970,2017-07-26,2020-11-24 11:24:52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2,140.0,90.0,9.0,65.0,20.0,0.0,0.00,2.20,3.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-26 11:24:52
41274,21979,2017-07-26,2020-11-24 09:27:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.9,140.0,90.0,9.0,65.0,72.0,201.0,0.90,8.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-26 09:27:16
41347,22052,2017-07-25,2020-11-24 18:31:40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.1,140.0,90.0,9.5,65.0,115.0,104.0,0.00,12.10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-25 18:31:40
41393,22098,2017-07-25,2020-11-24 10:39:46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4,140.0,90.0,9.0,65.0,58.0,181.0,0.60,6.40,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-25 10:39:46


In [ ]:
def remove_overlap(df):
  df_nooverlap = pd.DataFrame()
  for idx, row in df.iterrows():
    #if row["Date"] == pd.Timestamp("2017-07-25"):
    # adding buffer 5 mins to the time period; just to make sure
    df_test = df[ (df["timestamp"] > (row["timestamp"] + timedelta(hours=0, minutes=-35))) & (df["timestamp"] < (row["timestamp"] + timedelta(hours=2, minutes=5)))]
    if len(df_test) == 1:
      #print(row)
      df_nooverlap = df_nooverlap.append(df_test)
  return df_nooverlap

In [ ]:
meal_point_nooverlap = remove_overlap(meal_point)
meal_point_nooverlap

,Index,Date,Time,New Device Time,BG Reading (mg/dL),Basal Rate (U/h),Bolus Type,Bolus Volume Selected (U),Bolus Volume Delivered (U),Bolus Duration (h:mm:ss),Prime Type,Prime Volume Delivered (U),Alarm,Suspend,Rewind,BWZ Estimate (U),BWZ Target High BG (mg/dL),BWZ Target Low BG (mg/dL),BWZ Carb Ratio (g/U),BWZ Insulin Sensitivity (mg/dL/U),BWZ Carb Input (grams),BWZ BG Input (mg/dL),BWZ Correction Estimate (U),BWZ Food Estimate (U),BWZ Active Insulin (U),Sensor Calibration BG (mg/dL),Bolus Number,Bolus Cancellation Reason,BWZ Unabsorbed Insulin Total (U),Final Bolus Estimate,Scroll Step Size,Insulin Action Curve Time,Sensor Calibration Rejected Reason,Bolus Source,Network Device Associated Reason,Network Device Disassociated Reason,Network Device Disconnected Reason,timestamp
48,48,2018-02-12,2020-11-24 09:15:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,NaN,13.0,NaN,34.0,153.0,0.05,2.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 09:15:45
129,129,2018-02-12,2020-11-24 02:30:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2,NaN,NaN,12.0,NaN,15.0,0.0,0.00,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-12 02:30:55
188,188,2018-02-11,2020-11-24 20:33:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.8,NaN,NaN,8.0,NaN,71.0,0.0,0.00,8.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 20:33:18
207,207,2018-02-11,2020-11-24 18:14:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,115.0,80.0,8.0,100.0,8.0,0.0,0.00,1.00,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 18:14:37
261,261,2018-02-11,2020-11-24 12:43:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2,NaN,NaN,13.0,NaN,27.0,220.0,1.20,2.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-11 12:43:23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41164,21869,2017-07-27,2020-11-24 09:19:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,140.0,90.0,9.0,65.0,47.0,129.0,0.00,5.20,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-27 09:19:41
41172,21877,2017-07-27,2020-11-24 05:45:51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.8,140.0,90.0,11.0,80.0,24.0,201.0,0.70,2.10,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-27 05:45:51
41214,21919,2017-07-26,2020-11-24 19:15:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.2,140.0,90.0,9.5,65.0,60.0,86.0,-0.10,6.30,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-26 19:15:06
41261,21966,2017-07-26,2020-11-24 12:48:41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.4,140.0,90.0,8.5,65.0,63.0,255.0,1.70,7.40,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-07-26 12:48:41


In [ ]:
def extract_meal_data(df_meal, df):
  meal_data = pd.DataFrame()
  carb_value = []
  i = 0
  for idx, row in df_meal.iterrows():
    df_test = df[ (df["timestamp"] > (row["timestamp"] + timedelta(hours=0, minutes=-30))) & (df["timestamp"] < (row["timestamp"] + timedelta(hours=2, minutes=0)))]
    #print(len(df_test))
    if len(df_test) != 30:
      #print(len(df_test), row["timestamp"])
      continue
    meal_data = meal_data.append([df_test["Sensor Glucose (mg/dL)"].to_list()])
    carb_value.append(row["BWZ Carb Input (grams)"])
    i += 1
  print(i)
  meal_data["carb_val"] = carb_value
  return meal_data

In [ ]:
meal_data1 = extract_meal_data(meal_point_nooverlap, df_cgm)
meal_data1.dropna(inplace=True)
meal_data1

532


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,carb_val
0,150.0,162.0,174.0,180.0,178.0,184.0,185.0,194.0,193.0,186.0,182.0,174.0,172.0,167.0,166.0,164.0,165.0,166.0,167.0,164.0,159.0,160.0,158.0,158.0,153.0,154.0,151.0,150.0,149.0,150.0,34.0
0,177.0,194.0,200.0,201.0,189.0,162.0,166.0,173.0,176.0,169.0,167.0,166.0,168.0,180.0,182.0,178.0,171.0,160.0,145.0,138.0,132.0,120.0,127.0,129.0,132.0,123.0,104.0,100.0,107.0,106.0,71.0
0,208.0,215.0,212.0,206.0,203.0,196.0,190.0,180.0,184.0,180.0,177.0,186.0,194.0,205.0,207.0,220.0,200.0,194.0,196.0,198.0,200.0,201.0,211.0,215.0,217.0,216.0,221.0,221.0,219.0,209.0,27.0
0,202.0,197.0,191.0,185.0,197.0,206.0,209.0,207.0,174.0,168.0,168.0,142.0,136.0,135.0,136.0,147.0,148.0,137.0,137.0,144.0,141.0,135.0,133.0,126.0,110.0,104.0,74.0,62.0,58.0,57.0,10.0
0,106.0,114.0,136.0,155.0,163.0,169.0,162.0,158.0,153.0,146.0,140.0,127.0,116.0,110.0,106.0,108.0,115.0,121.0,128.0,132.0,133.0,126.0,100.0,82.0,70.0,65.0,60.0,59.0,59.0,61.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,200.0,198.0,191.0,183.0,175.0,172.0,157.0,147.0,140.0,132.0,127.0,129.0,137.0,131.0,127.0,120.0,112.0,108.0,99.0,87.0,77.0,70.0,58.0,55.0,57.0,57.0,53.0,46.0,40.0,40.0,54.0
0,152.0,153.0,165.0,173.0,177.0,179.0,183.0,189.0,196.0,200.0,204.0,213.0,211.0,212.0,203.0,192.0,179.0,169.0,162.0,149.0,130.0,123.0,122.0,125.0,124.0,129.0,133.0,137.0,141.0,145.0,47.0
0,209.0,210.0,209.0,210.0,210.0,213.0,216.0,212.0,213.0,210.0,205.0,203.0,192.0,179.0,172.0,169.0,168.0,170.0,176.0,181.0,183.0,188.0,194.0,201.0,201.0,199.0,200.0,204.0,210.0,212.0,24.0
0,81.0,77.0,74.0,67.0,70.0,72.0,74.0,75.0,71.0,67.0,66.0,69.0,77.0,89.0,112.0,123.0,132.0,130.0,125.0,121.0,112.0,100.0,87.0,83.0,71.0,60.0,40.0,40.0,40.0,40.0,60.0


# ground truth extraction (bins)

In [ ]:
min_carb1 = meal_data1["carb_val"].min()
print("min and max carb value for patient 1 : ", min_carb1, meal_data1["carb_val"].max())
print("cluster : ", (meal_data1["carb_val"].max() - min_carb1)/20)

min and max carb value for patient 1 :  3.0 120.0
cluster :  5.85


In [ ]:
def bin_values(val, min):
  return int((val-min)/20)

In [ ]:
meal_data1.sort_values(by=['carb_val'], inplace=True)
meal_data1["bin"] = meal_data1["carb_val"].apply(bin_values, min=min_carb1)
meal_data1.reset_index(inplace=True, drop=True)
meal_data1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,carb_val,bin
0,123.0,120.0,121.0,120.0,120.0,122.0,125.0,126.0,124.0,127.0,136.0,138.0,133.0,136.0,137.0,136.0,146.0,144.0,140.0,139.0,136.0,140.0,146.0,147.0,131.0,130.0,130.0,141.0,140.0,129.0,3.0,0
1,93.0,92.0,93.0,102.0,106.0,115.0,120.0,115.0,114.0,116.0,108.0,105.0,116.0,133.0,140.0,145.0,149.0,154.0,151.0,145.0,144.0,148.0,154.0,163.0,165.0,162.0,161.0,169.0,177.0,176.0,5.0,0
2,108.0,111.0,113.0,118.0,121.0,122.0,124.0,128.0,134.0,140.0,148.0,152.0,161.0,168.0,175.0,181.0,184.0,191.0,198.0,204.0,209.0,213.0,216.0,205.0,180.0,179.0,178.0,179.0,181.0,182.0,6.0,0
3,80.0,85.0,92.0,103.0,108.0,108.0,112.0,112.0,115.0,117.0,120.0,122.0,121.0,127.0,134.0,138.0,138.0,137.0,144.0,156.0,162.0,160.0,150.0,143.0,110.0,101.0,99.0,100.0,102.0,107.0,6.0,0
4,133.0,135.0,134.0,134.0,138.0,139.0,142.0,147.0,151.0,159.0,163.0,162.0,163.0,172.0,179.0,184.0,186.0,190.0,192.0,190.0,186.0,181.0,179.0,178.0,175.0,173.0,175.0,169.0,154.0,141.0,7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,137.0,131.0,128.0,127.0,127.0,128.0,130.0,131.0,128.0,123.0,121.0,118.0,116.0,114.0,117.0,121.0,121.0,127.0,110.0,97.0,92.0,86.0,90.0,98.0,105.0,111.0,105.0,103.0,118.0,126.0,100.0,4
420,81.0,85.0,95.0,106.0,125.0,144.0,150.0,150.0,146.0,143.0,145.0,149.0,148.0,154.0,158.0,163.0,169.0,161.0,150.0,137.0,123.0,116.0,111.0,112.0,117.0,121.0,125.0,131.0,134.0,142.0,100.0,4
421,177.0,176.0,181.0,189.0,195.0,204.0,205.0,201.0,195.0,198.0,205.0,206.0,220.0,227.0,235.0,242.0,255.0,262.0,272.0,280.0,289.0,299.0,317.0,324.0,340.0,349.0,365.0,378.0,389.0,395.0,102.0,4
422,148.0,152.0,155.0,151.0,147.0,145.0,158.0,162.0,162.0,159.0,162.0,167.0,170.0,169.0,161.0,155.0,157.0,163.0,169.0,180.0,190.0,198.0,201.0,194.0,192.0,182.0,167.0,155.0,144.0,135.0,112.0,5


# Feature extraction

In [ ]:
import scipy
import scipy.signal
import scipy.fftpack

In [ ]:
#velocity and acceleration
def get_velocity(df):
  df_vel = pd.DataFrame()
  vel_feat = pd.DataFrame()
  df_acc = pd.DataFrame()
  acc_feat = pd.DataFrame()
  #print(df.shape)
  for i in range(0, df.shape[1] - 1):
    df_vel['vel' + str(i)] = (df.iloc[:, i + 1] - df.iloc[:, i])

  for i in range(0, df_vel.shape[1] - 1):
    df_acc['acc' + str(i)] = (df_vel.iloc[:, i + 1] - df_vel.iloc[:, i])

  vel_feat["vel_min"] = df_vel.min(axis=1, skipna=True)
  vel_feat["vel_mean"] = df_vel.mean(axis=1, skipna=True)
  vel_feat["vel_max"] = df_vel.max(axis=1, skipna=True)

  acc_feat["acc_min"] = df_acc.min(axis=1, skipna=True)
  acc_feat["acc_mean"] = df_acc.mean(axis=1, skipna=True)
  acc_feat["acc_max"] = df_acc.max(axis=1, skipna=True)

  return pd.concat([vel_feat, acc_feat], axis=1)

In [ ]:
# PSD / periodogram
def get_psd_row(df):
  f, psd_dat = scipy.signal.periodogram(df.dropna())
  psd_row = [psd_dat[0:5].mean(), psd_dat[5:10].mean(), psd_dat[10:16].mean()]
  return psd_row


def get_psd(df):
  PSD = pd.DataFrame()
  PSD['vals'] = df.apply(lambda row: get_psd_row(row), axis=1)
  PSD_feat = pd.DataFrame(PSD.vals.tolist(), columns=['PSD_5', 'PSD_10', 'PSD_15'])

  return PSD_feat

In [ ]:
# IQR / inter quartile range
def get_iqr_row(df):
  iqrDat = scipy.stats.iqr(df.dropna())
  return iqrDat


def get_iqr(df):
  IQR_feat = pd.DataFrame()
  IQR_feat['IQR'] = df.apply(lambda x: get_iqr_row(x), axis=1)
  return IQR_feat

In [ ]:
# entropy
def get_entropy_row(df):
  df_counts = df.dropna().value_counts()
  entropy = scipy.stats.entropy(df_counts)
  return entropy


def get_entropy(df):
  entropy_feat = pd.DataFrame()
  entropy_feat['Entropy'] = df.apply(lambda x: get_entropy_row(x), axis=1)
  return entropy_feat


In [ ]:
# Fourier Transform
def get_fft_row(df):
  #print(df.to_list())
  
  FFT_vals = abs(scipy.fftpack.fft(df.dropna().to_list()))
  FFT_vals.sort()
  return np.flip(FFT_vals)[0:6]


def get_fft(df):
  FFT = pd.DataFrame()
  FFT['vals'] = df.apply(lambda row: get_fft_row(row), axis=1)
  FFT_feat = pd.DataFrame(FFT.vals.tolist(), columns=['FFT_1', 'FFT_2', 'FFT_3', 'FFT_4', 'FFT_5', 'FFT_6'])
  return FFT_feat

In [ ]:
# Time difference between CGM max and CGM min
# useless, doesn't change the result
def get_timediff(df):
  time_feat = pd.DataFrame()
  #time_feat["cgm_diff"] = df.max(axis=1, skipna=True) - df.min(axis=1, skipna=True)
  #time_feat["time_diff"] = int(vel_test.idxmax(axis=1)) - int(vel_test.idxmin(axis=1)) # this is literally = 4 for each value
  return time_feat

In [ ]:
########### features for all data

In [ ]:
X1 = pd.concat( [get_velocity(meal_data1.drop(columns=["carb_val", "bin"])),
get_psd(meal_data1.drop(columns=["carb_val", "bin"])),
get_iqr(meal_data1.drop(columns=["carb_val", "bin"])),
get_entropy(meal_data1.drop(columns=["carb_val", "bin"])),
get_fft(meal_data1.drop(columns=["carb_val", "bin"])),
get_timediff(meal_data1.drop(columns=["carb_val", "bin"]))],
axis = 1)
X1.drop(columns=["FFT_1", "FFT_3", "FFT_5"], inplace=True)
X1

,vel_min,vel_mean,vel_max,acc_min,acc_mean,acc_max,PSD_5,PSD_10,PSD_15,IQR,Entropy,FFT_2,FFT_4,FFT_6
0,-16.0,0.206897,11.0,-17.0,-0.285714,15.0,322.394422,88.168665,10.758539,14.50,2.904216,149.631461,61.489837,41.071560
1,-8.0,2.862069,17.0,-10.0,0.000000,14.0,3543.932793,427.101996,99.965454,39.75,3.170148,428.568429,223.323182,141.929475
2,-25.0,2.551724,9.0,-14.0,-0.071429,24.0,6278.568898,329.464515,107.199934,54.00,3.308778,663.268472,126.629299,112.467773
3,-33.0,0.931034,12.0,-26.0,0.000000,24.0,2646.462505,113.905411,32.143403,33.75,3.262568,395.180268,173.857085,94.574539
4,-15.0,0.275862,9.0,-9.0,-0.535714,8.0,2240.828645,26.706222,1.632055,35.75,3.123939,399.868304,68.190025,45.064866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,-17.0,-0.379310,15.0,-23.0,0.500000,17.0,1011.697318,59.541325,11.767797,20.75,2.932984,230.841598,109.732970,82.115178
420,-14.0,2.103448,19.0,-14.0,0.142857,8.0,2716.401853,193.460277,83.942670,31.75,3.245126,354.558844,253.638508,89.679776
421,-6.0,7.517241,18.0,-11.0,0.250000,13.0,25089.946408,2167.928300,806.437743,110.75,3.308778,1163.827349,512.566011,405.132531
422,-15.0,-0.448276,13.0,-10.0,-0.464286,15.0,1591.458282,72.558438,8.986067,14.75,3.089055,248.983895,206.714502,119.878447


In [ ]:
y1 = np.array(meal_data1["bin"])
y1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [ ]:
# scaling ------------------------------------
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize 

scaler = StandardScaler() 
X1_scaled = scaler.fit_transform(X1) 
  
# Normalizing the data so that  
# the data approximately follows a Gaussian distribution 
X1_normalized = normalize(X1_scaled) 
  
# Converting the numpy array into a pandas DataFrame 
X1_normalized = pd.DataFrame(X1_normalized)

X1_normalized

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.079996,0.089209,-0.091115,-0.101051,-0.138553,0.062165,-0.265576,-0.151286,-0.176310,-0.345198,-0.500730,-0.415531,-0.381257,-0.362833
1,0.570244,0.549167,0.202172,0.253895,0.145567,0.036960,-0.270844,0.016104,-0.102719,-0.249688,0.028653,-0.302582,-0.071708,-0.036274
2,-0.318322,0.432740,-0.226099,0.014020,0.046279,0.572576,-0.120236,-0.050367,-0.076017,-0.067562,0.398502,0.002808,-0.339853,-0.168689
3,-0.529261,0.165599,-0.057434,-0.454614,0.094990,0.433172,-0.204892,-0.145430,-0.156279,-0.210018,0.207218,-0.225843,-0.154386,-0.190750
4,0.136819,0.114498,-0.195466,0.233630,-0.405500,-0.252902,-0.248676,-0.215800,-0.224466,-0.222060,-0.086348,-0.253522,-0.439060,-0.417591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,0.044889,0.032800,0.052961,-0.318908,0.485318,0.138414,-0.241479,-0.162606,-0.172592,-0.294137,-0.438409,-0.344622,-0.277009,-0.221250
420,0.221542,0.401933,0.295333,0.015083,0.305678,-0.314730,-0.288257,-0.150487,-0.125308,-0.320986,0.244090,-0.370294,0.027226,-0.295826
421,0.212669,0.394030,0.081290,0.060758,0.145976,-0.008036,0.238938,0.419978,0.412980,0.188726,0.144330,0.204792,0.278786,0.433248
422,0.162411,0.037860,-0.026535,0.224741,-0.399987,0.088203,-0.322819,-0.225197,-0.253324,-0.487128,-0.199021,-0.474983,-0.112476,-0.137644


# Clustering

In [ ]:
from sklearn.cluster import KMeans, DBSCAN

km1 = KMeans(
    n_clusters=6, init='k-means++',
    n_init=100, max_iter=500, 
    tol=1e-04, random_state=0
)
y_km1 = km1.fit_predict(X1_normalized)
y_km1

array([2, 2, 0, 0, 2, 0, 5, 2, 4, 2, 5, 2, 5, 2, 2, 0, 0, 2, 5, 2, 1, 5,
       2, 2, 2, 3, 5, 5, 2, 2, 0, 5, 1, 1, 2, 0, 2, 0, 0, 4, 5, 2, 1, 3,
       2, 0, 2, 2, 2, 5, 2, 5, 0, 2, 2, 2, 1, 2, 2, 3, 0, 4, 2, 2, 4, 1,
       4, 1, 4, 3, 2, 1, 1, 2, 5, 1, 5, 5, 4, 0, 3, 2, 2, 2, 2, 5, 0, 2,
       3, 0, 1, 2, 3, 2, 5, 1, 3, 1, 5, 4, 4, 2, 2, 1, 1, 2, 3, 5, 5, 4,
       1, 5, 5, 1, 5, 4, 4, 5, 2, 2, 2, 2, 0, 4, 4, 4, 2, 5, 2, 5, 5, 0,
       1, 0, 2, 3, 3, 4, 2, 2, 3, 5, 2, 3, 2, 1, 0, 3, 4, 4, 0, 2, 1, 4,
       2, 2, 3, 2, 2, 2, 5, 3, 2, 1, 4, 2, 5, 3, 5, 1, 5, 3, 0, 0, 2, 0,
       2, 2, 3, 0, 1, 5, 4, 2, 3, 4, 2, 5, 2, 5, 5, 3, 0, 1, 0, 0, 2, 3,
       4, 3, 2, 3, 5, 4, 4, 2, 3, 1, 5, 3, 2, 5, 2, 1, 3, 2, 5, 2, 3, 0,
       4, 5, 5, 2, 5, 4, 0, 1, 5, 0, 5, 1, 0, 0, 3, 3, 1, 2, 4, 4, 3, 0,
       4, 1, 0, 2, 4, 4, 2, 3, 2, 0, 3, 3, 2, 2, 4, 2, 2, 5, 5, 3, 5, 3,
       4, 3, 0, 2, 1, 0, 0, 2, 2, 4, 0, 5, 3, 5, 4, 4, 2, 5, 5, 2, 5, 0,
       5, 1, 2, 2, 4, 3, 2, 3, 2, 3, 3, 5, 3, 3, 5,

In [ ]:
# dbscan
db_default = DBSCAN(eps=0.522, min_samples=3, metric='euclidean').fit(X1_normalized) 
y_db1 = db_default.labels_
y_db1

array([ 0,  0, -1, -1,  0, -1, -1,  0,  0,  0,  2,  0,  1,  0,  0, -1, -1,
        0,  0,  0,  1,  0,  0,  0,  0, -1, -1, -1,  0,  0, -1, -1,  1,  3,
        0,  0,  0, -1,  4,  0, -1,  0, -1,  2,  0, -1,  0,  0,  0, -1,  0,
       -1, -1,  0,  0,  0,  3,  0,  0, -1, -1,  0, -1,  0,  0, -1, -1,  1,
        2,  2,  0,  1,  1,  0,  0,  1, -1,  0,  0,  4,  2,  0,  0,  0,  0,
        0, -1,  0,  2, -1,  1,  0,  2,  0,  1, -1, -1,  3, -1,  0, -1,  0,
        0,  3,  1,  0,  2,  0,  0,  0,  1,  0,  0,  1, -1,  0,  0, -1,  0,
        0,  0,  0, -1,  0,  0,  0,  0, -1,  0, -1, -1,  4, -1,  4,  0, -1,
       -1,  0,  0,  0, -1, -1,  0,  2, -1,  1, -1,  2,  0,  0, -1,  0, -1,
       -1,  0,  0, -1,  0,  5,  0, -1,  2,  0,  1,  0,  0,  0,  2,  6,  1,
       -1, -1, -1, -1,  0,  5,  0,  0,  2, -1, -1, -1,  0, -1, -1, -1,  0,
        0, -1, -1,  0,  2, -1,  3, -1, -1,  0, -1, -1, -1, -1, -1,  1, -1,
        0,  0,  2,  1,  0, -1,  0, -1, -1,  1,  2,  0, -1,  0,  2, -1, -1,
       -1,  0, -1,  0, -1

In [ ]:
np.unique(y_db1)

array([-1,  0,  1,  2,  3,  4,  5,  6])

# Metrics

In [ ]:
## PURITY

from sklearn import metrics

def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 

In [ ]:
p_km = purity_score(y1, y_km1)

In [ ]:
p_db = purity_score(y1, y_db1)

In [ ]:
# SSE

In [ ]:
sse_km = km1.inertia_

In [ ]:
import copy
db_df = copy.deepcopy(X1_normalized)
db_df["db"] = y_db1
db_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,db
0,0.079996,0.089209,-0.091115,-0.101051,-0.138553,0.062165,-0.265576,-0.151286,-0.176310,-0.345198,-0.500730,-0.415531,-0.381257,-0.362833,0
1,0.570244,0.549167,0.202172,0.253895,0.145567,0.036960,-0.270844,0.016104,-0.102719,-0.249688,0.028653,-0.302582,-0.071708,-0.036274,0
2,-0.318322,0.432740,-0.226099,0.014020,0.046279,0.572576,-0.120236,-0.050367,-0.076017,-0.067562,0.398502,0.002808,-0.339853,-0.168689,-1
3,-0.529261,0.165599,-0.057434,-0.454614,0.094990,0.433172,-0.204892,-0.145430,-0.156279,-0.210018,0.207218,-0.225843,-0.154386,-0.190750,-1
4,0.136819,0.114498,-0.195466,0.233630,-0.405500,-0.252902,-0.248676,-0.215800,-0.224466,-0.222060,-0.086348,-0.253522,-0.439060,-0.417591,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,0.044889,0.032800,0.052961,-0.318908,0.485318,0.138414,-0.241479,-0.162606,-0.172592,-0.294137,-0.438409,-0.344622,-0.277009,-0.221250,0
420,0.221542,0.401933,0.295333,0.015083,0.305678,-0.314730,-0.288257,-0.150487,-0.125308,-0.320986,0.244090,-0.370294,0.027226,-0.295826,0
421,0.212669,0.394030,0.081290,0.060758,0.145976,-0.008036,0.238938,0.419978,0.412980,0.188726,0.144330,0.204792,0.278786,0.433248,1
422,0.162411,0.037860,-0.026535,0.224741,-0.399987,0.088203,-0.322819,-0.225197,-0.253324,-0.487128,-0.199021,-0.474983,-0.112476,-0.137644,0


In [ ]:
db_centers = db_df.groupby(["db"]).mean().reset_index()
db_centers

,db,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-1,-0.102372,-0.051098,0.022514,-0.068338,0.039704,0.045849,-0.060399,-0.094267,-0.092470,-0.036018,0.078318,-0.005658,0.008826,-0.043704
1,0,0.186964,0.037772,-0.143347,0.186913,0.009803,-0.184085,-0.219672,-0.166693,-0.170081,-0.229113,-0.162460,-0.246269,-0.216848,-0.222338
2,1,0.142683,0.393835,0.184097,0.052795,-0.112753,-0.047430,0.299744,0.210878,0.223568,0.287978,0.189890,0.310306,0.182624,0.251231
3,2,-0.157711,-0.302898,-0.104000,-0.005226,-0.072340,0.015131,0.263593,0.199260,0.204768,0.322630,0.122705,0.301101,0.181120,0.232556
4,3,0.446392,0.529974,0.009331,0.215846,0.122514,-0.213412,0.031268,0.189307,0.191413,0.068003,0.202602,0.071909,0.161529,0.242220
5,4,-0.367172,0.024686,0.176755,-0.521406,0.004381,0.487789,-0.124551,-0.068938,-0.093636,-0.111215,-0.098961,-0.136653,-0.114776,-0.013369
6,5,0.188087,0.062620,0.438239,0.029615,0.138093,0.196190,-0.320337,-0.256342,-0.247083,-0.365231,0.070168,-0.440598,-0.168341,-0.049432
7,6,0.207548,0.182028,0.393712,0.090331,-0.140623,-0.042330,0.008168,-0.227943,-0.218746,0.146570,0.376827,0.180746,-0.430229,-0.399841


In [ ]:
db_centers.loc[db_centers['db'] == -1]

,db,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-1,-0.102372,-0.051098,0.022514,-0.068338,0.039704,0.045849,-0.060399,-0.094267,-0.09247,-0.036018,0.078318,-0.005658,0.008826,-0.043704


In [ ]:
squared_err = 0
for idx, row in db_df.iterrows():
  squared_err += (db_centers.loc[db_centers['db'] == -1] - row)**2
sse_db = float(squared_err.drop(columns=["db"]).sum(axis=1))

In [ ]:
## ENTROPY
from scipy.stats import entropy
from collections import Counter

Counter(y1).keys() # equals to list(set(y1))
Counter(y1).values() 

dict_values([109, 125, 106, 54, 28, 2])

In [ ]:
Counter(y1).keys()

dict_keys([0, 1, 2, 3, 4, 5])

In [ ]:
counter1 = Counter(y1)

In [ ]:
counter1

Counter({0: 109, 1: 125, 2: 106, 3: 54, 4: 28, 5: 2})

In [ ]:
prev_v = -1
total_entropy_km1 = 0
for k,v in counter1.items():
  #print(k,v)
  counter2 = Counter(y_km1[prev_v+1:prev_v+v+1])
  #print(counter2)
  total_entropy_km1 += (v/len(y1)) * entropy(np.array(list(counter2.values()))/v)
  prev_v = v
total_entropy_km1

1.6946097213065405

In [ ]:
prev_v = -1
total_entropy_db1 = 0
for k,v in counter1.items():
  #print(k,v)
  counter2 = Counter(y_db1[prev_v+1:prev_v+v+1])
  del counter2[-1]
  #print(counter2)
  total_entropy_db1 += (v/len(y1)) * entropy(np.array(list(counter2.values()))/v)
  prev_v = v
total_entropy_db1

1.12697250181442

# Final results

In [ ]:
result = pd.DataFrame(columns = ["SSE for Kmeans",	"SSE for DBSCAN",	"Entropy for Kmeans",	"Entropy for DBSCAN",	"Purity for K means",	"Purity for DBSCAN"])
result_length = len(result)
result.loc[result_length] = [sse_km,sse_db,total_entropy_km1,total_entropy_db1,p_km,p_db]

In [ ]:
result

,SSE for Kmeans,SSE for DBSCAN,Entropy for Kmeans,Entropy for DBSCAN,Purity for K means,Purity for DBSCAN
0,209.037641,435.998712,1.69461,1.126973,0.339623,0.349057


In [ ]:
result.to_csv("Results.csv", index=False)